In [7]:
import numpy as np
import pandas as pd

In [8]:
winedat = 'winemag-data-130k-v2.csv'
allwine = pd.read_csv(winedat, low_memory=False)
allwine.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [9]:
allwine.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [10]:
usewine = allwine.drop(columns=['Unnamed: 0','description','designation','taster_name','taster_twitter_handle'])
usewine.head()

,country,points,price,province,region_1,region_2,title,variety,winery
0,Italy,87,NaN,Sicily & Sardinia,Etna,NaN,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,87,15.0,Douro,NaN,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [11]:
len(usewine['variety'].unique())

708

In [12]:
usewine['points'].unique()

array([ 87,  86,  85,  88,  92,  91,  90,  89,  83,  82,  81,  80, 100,
        98,  97,  96,  95,  93,  94,  84,  99], dtype=int64)

In [13]:
usewine['country'].unique()

array(['Italy', 'Portugal', 'US', 'Spain', 'France', 'Germany',
       'Argentina', 'Chile', 'Australia', 'Austria', 'South Africa',
       'New Zealand', 'Israel', 'Hungary', 'Greece', 'Romania', 'Mexico',
       'Canada', nan, 'Turkey', 'Czech Republic', 'Slovenia',
       'Luxembourg', 'Croatia', 'Georgia', 'Uruguay', 'England',
       'Lebanon', 'Serbia', 'Brazil', 'Moldova', 'Morocco', 'Peru',
       'India', 'Bulgaria', 'Cyprus', 'Armenia', 'Switzerland',
       'Bosnia and Herzegovina', 'Ukraine', 'Slovakia', 'Macedonia',
       'China', 'Egypt'], dtype=object)

In [14]:
winecounts = usewine['variety'].value_counts()

In [15]:
winecounts[0:12]

Pinot Noir                  13272
Chardonnay                  11753
Cabernet Sauvignon           9472
Red Blend                    8946
Bordeaux-style Red Blend     6915
Riesling                     5189
Sauvignon Blanc              4967
Syrah                        4142
Rosé                         3564
Merlot                       3102
Nebbiolo                     2804
Zinfandel                    2714
Name: variety, dtype: int64

In [16]:
eightvars = ['Pinot Noir','Chardonnay','Cabernet Sauvignon','Riesling','Sauvignon Blanc','Syrah','Rosé','Merlot']

In [17]:
mostdata_wines = usewine[usewine['variety'].isin(eightvars)]
mostdata_wines.head()

,country,points,price,province,region_1,region_2,title,variety,winery
3,US,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
10,US,87,19.0,California,Napa Valley,Napa,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
12,US,87,34.0,California,Alexander Valley,Sonoma,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini
14,US,87,12.0,California,Central Coast,Central Coast,Mirassou 2012 Chardonnay (Central Coast),Chardonnay,Mirassou


In [18]:
len(mostdata_wines)

55461

In [19]:
reg1_count = len(mostdata_wines['country'].dropna())
reg1_count

55446

In [20]:
mostdata_wines.columns

Index(['country', 'points', 'price', 'province', 'region_1', 'region_2',
       'title', 'variety', 'winery'],
      dtype='object')

In [21]:
wine_review_df = mostdata_wines.drop(columns=['price','province','region_1','region_2','title'])
wine_review_df.head()

,country,points,variety,winery
3,US,87,Riesling,St. Julian
4,US,87,Pinot Noir,Sweet Cheeks
10,US,87,Cabernet Sauvignon,Kirkland Signature
12,US,87,Cabernet Sauvignon,Louis M. Martini
14,US,87,Chardonnay,Mirassou


In [22]:
#make searchterm column
wine_review_df['winery_search'] = wine_review_df['winery'] + " winery" + " " + wine_review_df['country']
wine_review_df.head()

,country,points,variety,winery,winery_search
3,US,87,Riesling,St. Julian,St. Julian winery US
4,US,87,Pinot Noir,Sweet Cheeks,Sweet Cheeks winery US
10,US,87,Cabernet Sauvignon,Kirkland Signature,Kirkland Signature winery US
12,US,87,Cabernet Sauvignon,Louis M. Martini,Louis M. Martini winery US
14,US,87,Chardonnay,Mirassou,Mirassou winery US


In [23]:
len(wine_review_df)

55461

In [37]:
# save cleaned file to csv 
wine_review_df.to_csv('wine_reviews_cleaned.csv')

In [24]:
#create list of winery names to search with google maps function
winery_list = wine_review_df.winery_search.unique()
len(winery_list)

8701

In [25]:
US_winery_list = wine_review_df[wine_review_df['country'] == 'US']
US_winery_list.head()

,country,points,variety,winery,winery_search
3,US,87,Riesling,St. Julian,St. Julian winery US
4,US,87,Pinot Noir,Sweet Cheeks,Sweet Cheeks winery US
10,US,87,Cabernet Sauvignon,Kirkland Signature,Kirkland Signature winery US
12,US,87,Cabernet Sauvignon,Louis M. Martini,Louis M. Martini winery US
14,US,87,Chardonnay,Mirassou,Mirassou winery US


In [26]:
len(US_winery_list)

34379

In [28]:
# Dependencies
import json
import requests
import logging
import time

logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

# Google developer API key
from config import g_key

In [29]:
return_full_results = False

def get_googlemaps_results(winery, api_key=None, return_full_response=False):
    
    params = {"key": api_key, "input": winery, "inputtype": "textquery", "fields": "geometry"}

    # Set Geocoding url
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
        
    # Ping google for the results:
    response = requests.get(base_url, params=params)
    results = response.json()
    
    # if there are no results or an error, return empty results.

    if len(results['candidates']) == 0:
        output = {
            "latitude": None,
            "longitude": None,
        }
    else:    
        answer = results['candidates'][0]
        output = {
            "winery_search_term": winery,
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
        }
        
    # Append other details:    
    output['number_of_results'] = len(results['candidates'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    
    return output

In [30]:
test_result = get_googlemaps_results("St. Julian winery US", g_key, return_full_results)
test_result

{'winery_search_term': 'St. Julian winery US',
 'latitude': 42.2122513,
 'longitude': -85.8917127,
 'number_of_results': 1,
 'status': 'OK'}

In [31]:
test_winery_list = winery_list[0:10]
test_winery_list

array(['St. Julian winery US', 'Sweet Cheeks winery US',
       'Kirkland Signature winery US', 'Louis M. Martini winery US',
       'Mirassou winery US', 'Richard Böcking winery Germany',
       'Acrobat winery US', 'Bianchi winery US',
       'Castello di Amorosa winery US', 'Envolve winery US'], dtype=object)

In [36]:
#create empty list that we will populate with for loop
winery_latlongs = []

#set output file name
output_filename = 'winery_latlong_data.csv'

# Go through each address in turn
for winery in winery_list:
    # While the address geocoding is not finished:
    geocoded = False
    while geocoded is not True:
        # Geocode the address with google
        try:
            geocode_result = get_googlemaps_results(winery, g_key, return_full_response=return_full_results)
        except Exception as e:
            logger.exception(e)
            logger.error("Major error with {}".format(winery))
            logger.error("Skipping!")
            geocoded = True
            
        # If we're over the API limit, backoff for a while and try again later.
        if geocode_result['status'] == 'OVER_QUERY_LIMIT':
            logger.info("Hit Query Limit! Backing off for a bit.")
            time.sleep(BACKOFF_TIME * 60) # sleep for 30 minutes
            geocoded = False
        else:
            # If we're ok with API use, save the results
            # Note that the results might be empty / non-ok - log this
            if geocode_result['status'] != 'OK':
                logger.warning("Error geocoding {}: {}".format(winery, geocode_result['status']))
            logger.debug("Geocoded: {}: {}".format(winery, geocode_result['status']))
            winery_latlongs.append(geocode_result)           
            geocoded = True

    # Print status every 100 addresses
    if len(winery_latlongs) % 100 == 0:
    	print("Completed {} of {} address".format(len(winery_latlongs), len(winery_list)))
            
    # Every 500 addresses, save progress to file(in case of a failure so we have something!)
    if len(winery_latlongs) % 500 == 0:
        pd.DataFrame(winery_latlongs).to_csv("{}_bak".format(output_filename))

# All done
print("Finished geocoding all wineries")

# save fulls results to CSV
pd.DataFrame(winery_latlongs).to_csv(output_filename)

Geocoded: St. Julian winery US: OK
Geocoded: St. Julian winery US: OK
Geocoded: Sweet Cheeks winery US: OK
Geocoded: Sweet Cheeks winery US: OK
Geocoded: Kirkland Signature winery US: OK
Geocoded: Kirkland Signature winery US: OK
Geocoded: Louis M. Martini winery US: OK
Geocoded: Louis M. Martini winery US: OK
Error geocoding Mirassou winery US: ZERO_RESULTS
Error geocoding Mirassou winery US: ZERO_RESULTS
Geocoded: Mirassou winery US: ZERO_RESULTS
Geocoded: Mirassou winery US: ZERO_RESULTS
Geocoded: Richard Böcking winery Germany: OK
Geocoded: Richard Böcking winery Germany: OK
Error geocoding Acrobat winery US: ZERO_RESULTS
Error geocoding Acrobat winery US: ZERO_RESULTS
Geocoded: Acrobat winery US: ZERO_RESULTS
Geocoded: Acrobat winery US: ZERO_RESULTS
Geocoded: Bianchi winery US: OK
Geocoded: Bianchi winery US: OK
Geocoded: Castello di Amorosa winery US: OK
Geocoded: Castello di Amorosa winery US: OK
Geocoded: Envolve winery US: OK
Geocoded: Envolve winery US: OK
Geocoded: Erath wi

Completed 100 of 8701 address


Geocoded: Cambria winery US: OK
Geocoded: Cambria winery US: OK
Geocoded: J. Christopher winery US: OK
Geocoded: J. Christopher winery US: OK
Geocoded: Salomon-Undhof winery Austria: OK
Geocoded: Salomon-Undhof winery Austria: OK
Geocoded: Talley winery US: OK
Geocoded: Talley winery US: OK
Geocoded: Parallel winery US: OK
Geocoded: Parallel winery US: OK
Geocoded: Ventisquero winery Chile: OK
Geocoded: Ventisquero winery Chile: OK
Geocoded: Aquinas winery US: OK
Geocoded: Aquinas winery US: OK
Error geocoding Arboleda winery Chile: ZERO_RESULTS
Error geocoding Arboleda winery Chile: ZERO_RESULTS
Geocoded: Arboleda winery Chile: ZERO_RESULTS
Geocoded: Arboleda winery Chile: ZERO_RESULTS
Geocoded: Buried Cane winery US: OK
Geocoded: Buried Cane winery US: OK
Geocoded: Cramele Recas winery Romania: OK
Geocoded: Cramele Recas winery Romania: OK
Geocoded: Domaine Bertagna winery France: OK
Geocoded: Domaine Bertagna winery France: OK
Geocoded: Otto's Constant Dream winery New Zealand: OK
G

Geocoded: Limerick Lane winery US: OK


Completed 200 of 8701 address


...

Geocoded: H. Blin winery France: OK
Geocoded: H. Blin winery France: OK
Geocoded: We Are Sonoma winery US: OK
Geocoded: We Are Sonoma winery US: OK
Geocoded: Parker winery Australia: OK
Geocoded: Parker winery Australia: OK
Geocoded: Le Vigne di Ca' Nova winery Italy: OK
Geocoded: Le Vigne di Ca' Nova winery Italy: OK
Geocoded: Domaine Perzinsky winery France: OK
Geocoded: Domaine Perzinsky winery France: OK
Geocoded: Marchand & Burch winery Australia: OK
Geocoded: Marchand & Burch winery Australia: OK
Error geocoding Adolph Mueller winery Germany: ZERO_RESULTS
Error geocoding Adolph Mueller winery Germany: ZERO_RESULTS
Geocoded: Adolph Mueller winery Germany: ZERO_RESULTS
Geocoded: Adolph Mueller winery Germany: ZERO_RESULTS
Geocoded: Château de Landué winery France: OK
Geocoded: Château de Landué winery France: OK
Geocoded: Maple Ridge winery US: OK
Geocoded: Maple Ridge winery US: OK
Geocoded: Old House winery US: OK
Geocoded: Old House winery US: OK
Geocoded: Bully Hill winery US: 

Geocoded: Sundae wines winery US: OK
Geocoded: Sundae wines winery US: OK
Geocoded: Château d'Haurets winery France: OK
Geocoded: Château d'Haurets winery France: OK
Geocoded: Tenuta Montecchiesi winery Italy: OK
Geocoded: Tenuta Montecchiesi winery Italy: OK
Geocoded: Mumm Napa winery US: OK
Geocoded: Mumm Napa winery US: OK
Geocoded: Makana winery US: OK
Geocoded: Makana winery US: OK
Geocoded: Château de Garcinières winery France: OK
Geocoded: Château de Garcinières winery France: OK
Geocoded: Lyrarakis winery Greece: OK
Geocoded: Lyrarakis winery Greece: OK
Geocoded: Tuxedo Cellars winery US: OK
Geocoded: Tuxedo Cellars winery US: OK
Geocoded: Jaillance winery France: OK
Geocoded: Jaillance winery France: OK
Geocoded: OZV winery US: OK
Geocoded: OZV winery US: OK
Geocoded: Ferguson Crest winery US: OK
Geocoded: Ferguson Crest winery US: OK
Geocoded: Carmel Hills Winery winery US: OK
Geocoded: Carmel Hills Winery winery US: OK
Error geocoding Taking Root winery Moldova: ZERO_RESULTS

Completed 8700 of 8701 address


Geocoded: Mas de Pampelonne winery France: OK
Geocoded: Mas de Pampelonne winery France: OK


Finished geocoding all wineries
